Naive Ensemble

In [12]:
!pip install transformers

In [13]:
import pandas as pd
import ast
from transformers import BertTokenizerFast,TFBertModel
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
from keras.callbacks import EarlyStopping,ModelCheckpoint

In [16]:
root_path=""

In [68]:
mode="test"

In [69]:
if mode=="validation":
  file_path=root_path+"validation.csv"
  file_path_BIGRU=root_path+"Spanbert_TC_BIGRU_validation.csv"
  file_path_conv=root_path+"Spanbert_TC_Conv_validation.csv"
  file_path_tc=root_path+"Spanbert_TC_validation.csv"
  final_file_path=root_path+f"All_results_{mode}.csv"
else:
  file_path="https://raw.github.com/ipavlopoulos/toxic_spans/master/data/tsd_test.csv"
  file_path_BIGRU=root_path+"Spanbert_TC_BIGRU_test.csv"
  file_path_conv=root_path+"Spanbert_TC_Conv_test.csv"
  file_path_tc=root_path+"Spanbert_TC_test.csv"
  final_file_path=root_path+f"All_results_{mode}.csv"

In [70]:
#test_data=https://raw.githubusercontent.com/nangokosu/Toxic_span_detection/main/spanBERT_TC/validation.csv?token=ANVHFMCFBYSQQPTBS5ATGOLASW6AK
#validation_data="https://raw.githubusercontent.com/nangokosu/Toxic_span_detection/main/spanBERT_TC/validation.csv?token=ANVHFMCFBYSQQPTBS5ATGOLASW6AK"
df=pd.read_csv(file_path)[["spans","text"]]
df["span"]=df.spans.apply(lambda x: ast.literal_eval(x))

In [71]:
df_BIGRU=pd.read_csv(file_path_BIGRU)[["span","text"]]
df_BIGRU.span=df_BIGRU.span.apply(lambda x: ast.literal_eval(x))

In [72]:
df_conv=pd.read_csv(file_path_conv)[["span","text"]]
df_conv.span=df_conv.span.apply(lambda x: ast.literal_eval(x))

In [73]:
df_tc=pd.read_csv(file_path_tc)[["span","text"]]
df_tc.span=df_tc.span.apply(lambda x: ast.literal_eval(x))

In [74]:
df_methods=df_conv.merge(df_BIGRU,how="inner",left_on="text",right_on="text")

In [75]:
df_methods["correct_span"]=df.span

In [76]:
def find_intersection(x,y):
  set_1=set(x)
  set_2=set(y)
  intersection=set_1.intersection(set_2)
  return list(intersection)

In [77]:
def find_union(x,y):
  set_1=set(x)
  set_2=set(y)
  intersection=set_1.union(set_2)
  return list(intersection)

In [78]:
df_methods["span_intersection"]=df_methods[['span_x','span_y']].apply(lambda x: find_intersection(x[0],x[1]),axis=1)
df_methods["span_union"]=df_methods[['span_x','span_y']].apply(lambda x: find_union(x[0],x[1]),axis=1)
df_methods["span_TC"]=df_tc.span

In [79]:
def f1(predictions, gold):
    """
    F1 (a.k.a. DICE) operating on two lists of offsets (e.g., character).
    >>> assert f1([0, 1, 4, 5], [0, 1, 6]) == 0.5714285714285714
    :param predictions: a list of predicted offsets
    :param gold: a list of offsets serving as the ground truth
    :return: a score between 0 and 1
    """
    if len(gold) == 0:
        return 1. if len(predictions) == 0 else 0.
    if len(predictions) == 0:
        return 0.
    predictions_set = set(predictions)
    gold_set = set(gold)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)

In [80]:
df_methods["f1_intersection"]=df_methods[["span_intersection","correct_span"]].apply(lambda x: f1(x[0],x[1]),axis=1)
df_methods["f1_conv"]=df_methods[["span_x","correct_span"]].apply(lambda x: f1(x[0],x[1]),axis=1)
df_methods["f1_BIGRU"]=df_methods[["span_y","correct_span"]].apply(lambda x: f1(x[0],x[1]),axis=1)
df_methods["f1_union"]=df_methods[["span_union","correct_span"]].apply(lambda x: f1(x[0],x[1]),axis=1)
df_methods["f1_TC"]=df_methods[["span_TC","correct_span"]].apply(lambda x: f1(x[0],x[1]),axis=1)

In [95]:
print(df_methods["f1_intersection"].mean())
print(df_methods["f1_conv"].mean())
print(df_methods["f1_BIGRU"].mean())
print(df_methods["f1_union"].mean())
print(df_methods["f1_TC"].mean())

0.7854275734207472
0.8090226837052209
0.8184481137819403
0.8330105925622171
0.8424839311239191


In [81]:
def find_spans(span_index,text):
  '''Function takes in the span index list and returns the text those indices correspond to'''
  try:  
    span_index=[int(item) for item in span_index]
    current_index=span_index[0]
    spans=[]
    span_text=[]
    for i in range(len(span_index)-1):
      if span_index[i+1]-span_index[i]>1:
        spans.append((current_index,span_index[i]+1))
        current_index=span_index[i+1]
    if current_index==span_index[0]:
      spans.append((current_index,span_index[-1]+1))
    return spans
  except:
    return []

In [82]:
spanbert_path='/content/drive/MyDrive/Span NN materials/spanbert_hf_base/'

In [83]:
tokenizer = BertTokenizerFast.from_pretrained(spanbert_path)

In [84]:
class TweetResults:
  def __init__(self,text,span_index,max_len,doc_max_len):
    self.text=text
    self.span_index=span_index
    self.max_len=max_len
    self.doc_max_len=doc_max_len
  
  def process(self):
    text=self.text
    span_index=self.span_index
    character_in_ans=[0]*len(text)
    spans=find_spans(span_index,text)
    for start,end in spans: # marking the 1s
      for idx in range(start,end):
        character_in_ans[idx]=1

    # Generates tokenized text
    tokenized_text = tokenizer.encode_plus(text, return_offsets_mapping=True, max_length = self.max_len)

    # Generate the actual tokens:
    ans_token_idx = []
    for idx, (start, end) in enumerate(tokenized_text.offset_mapping):
        if sum(character_in_ans[start:end]) > 0:
            ans_token_idx.append([0,1])
        else: 
            ans_token_idx.append([1,0])

    # Padding
    padding_length_toxic=self.doc_max_len - len(ans_token_idx)
    if padding_length_toxic>0:
      #character_in_ans=character_in_ans+[0]*padding_length_toxic
      ans_token_idx=ans_token_idx+[[0,0]]*padding_length_toxic
    
    self.ans_token_idx=ans_token_idx
    self.context_token_to_char = tokenized_text.offset_mapping

In [85]:
def create_tweet_examples(data,max_len,doc_max_len):
    tweet_examples = []
    count = 0
    for index, row in data.iterrows():
        span_index = row['span']
        text=row['text']
        ## turn the data into TrainTweetExample objects
        try:
          tweet_eg = TweetResults(text, span_index,max_len,doc_max_len)
          tweet_eg.process()
          tweet_examples.append(tweet_eg)
        except: ## keep track of the number that can't be tokenized/processed
            count += 1 
    print("Couldn't process",count,"points")
    return tweet_examples

In [86]:
max_len=400
doc_max_len=400

In [87]:
BIGRU_data=create_tweet_examples(df_BIGRU,max_len,doc_max_len)
conv_data=create_tweet_examples(df_conv,max_len,doc_max_len)
correct_data=create_tweet_examples(df,max_len,doc_max_len)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Couldn't process 0 points
Couldn't process 0 points
Couldn't process 0 points


In [88]:
import numpy as np
BIGRU_data_prediction=np.array([element.ans_token_idx for element in BIGRU_data])[:,:,1].reshape(-1,400,1)
conv_data_prediction=np.array([element.ans_token_idx for element in conv_data])[:,:,1].reshape(-1,400,1)
correct_data_prediction=np.array([element.ans_token_idx for element in correct_data])

In [53]:
def create_model(max_len,doc_max_len):
  input_BIGRU=layers.Input(shape=(400,1))
  input_conv=layers.Input(shape=(400,1))
  #input_tc=layers.Input(shape=(400,1))
  concat=layers.Concatenate()([input_BIGRU,input_conv])
  #concat=layers.TimeDistributed(layers.Dense(1,activation="relu"))(concat)
  concat=layers.TimeDistributed(layers.Dense(2,activation="softmax"))(concat)
  return Model([input_BIGRU,input_conv],concat)

In [54]:
model=create_model(max_len,doc_max_len)

In [55]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 400, 1)]     0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, 400, 1)]     0                                            
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 400, 2)       0           input_8[0][0]                    
                                                                 input_9[0][0]                    
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 400, 2)       6           concatenate_3[0][0]        

In [56]:
early_stop=EarlyStopping(monitor="val_accuracy",patience=10,restore_best_weights=True)
model_check=ModelCheckpoint(root_path+"Ensemble_model",monitor="val_accuracy",save_best_only=True)

In [57]:
model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [58]:
model.fit([BIGRU_data_prediction,conv_data_prediction],correct_data_prediction,epochs=200,validation_split=0.10,callbacks=[early_stop,model_check])

Epoch 1/200
23/23 [==============================] - 1s 17ms/step - loss: 0.0864 - accuracy: 0.9928 - val_loss: 0.0987 - val_accuracy: 0.9893
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Span NN materials/Ensemble_model/assets
Epoch 2/200
23/23 [==============================] - 0s 9ms/step - loss: 0.0852 - accuracy: 0.9900 - val_loss: 0.0961 - val_accuracy: 0.9890
Epoch 3/200
23/23 [==============================] - 0s 8ms/step - loss: 0.0878 - accuracy: 0.9904 - val_loss: 0.0936 - val_accuracy: 0.9890
Epoch 4/200
23/23 [==============================] - 0s 9ms/step - loss: 0.0786 - accuracy: 0.9912 - val_loss: 0.0912 - val_accuracy: 0.9890
Epoch 5/200
23/23 [==============================] - 0s 9ms/step - loss: 0.0795 - accuracy: 0.9894 - val_loss: 0.0889 - val_accuracy: 0.9890
Epoch 6/200
23/23 [==============================] - 0s 8ms/step - loss: 0.0768 - accuracy: 0.9905 - val_loss: 0.0867 - val_accuracy: 0.9890
Epoch 7/200
23/23 [==============================] - 0s

In [89]:
def get_indices(x):
  indice_prediction=[]
  for single_prediction in x:
    indice_prediction.append([idx for idx,value in enumerate(single_prediction) if value[1]>=0.5])
  return indice_prediction

In [90]:
def get_indices_final(indice_prediction,tweet_examples):
  all_correct_indices=[]
  for i in range(len(indice_prediction)):
    correct_indices=[]
    indices=indice_prediction[i]
    tweet=tweet_examples[i].context_token_to_char
    #assert len(indices)>len(tweet)
    for i in indices:
      if i <len(tweet):
        start,end=tweet[i]
        correct_indices.extend([index for index in range(start,end)])
    all_correct_indices.append(correct_indices)
  return all_correct_indices

In [91]:
def all_f1_scores(predictions,golds):
  f1_scores=[]
  for i in range(len(predictions)):
    f1_score=f1(predictions[i],golds.span.values[i])
    f1_scores.append(f1_score)
  return f1_scores

In [92]:
def convert_to_indices(input_data,tweet_examples):
  '''
  Input data takes the form of a list of two arrays.
  '''
  global model
  predictions=model.predict(input_data)
  indice_prediction=get_indices(predictions)
  final_indices=get_indices_final(indice_prediction,tweet_examples)
  return final_indices

In [93]:
indices_pred=convert_to_indices([BIGRU_data_prediction,conv_data_prediction],BIGRU_data)
df_methods["f1_ensemble"]=all_f1_scores(indices_pred,df)

In [94]:
print(np.mean(df_methods["f1_ensemble"]))
print(np.mean(df_methods["f1_TC"]))

0.8088108866903423
0.8424839311239191


In [96]:
df_methods.to_csv(final_file_path,index=False)